In [1]:
from utils import * 
from algorithms import *
from agents import *

In [2]:
numRounds, numPeriods, numSteps = 1, 10000, 10
numBuyers, numSellers, numTokens = 4, 4, 4
gameTypes, seed = '1111', 42
disclosure = ['bids', 'asks', 'currentBid', 'currentBidIdx','currentAsk', 'currentAskIdx', 'buy', 'sell', 'price', 'sale']
depth = 1
gameData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed]
numStates = 18
gamma=0.99
lr_mu = 0.00003
lr_q = 0.0003
tau=0.005
batch_size = 32
buffer_limit = 50000
wait_period = 2000
grad_steps = 1
reward_norm = False
std_init = 0.3
std_decay = 0.9995
std_min = 0.02
buyers = [Reinforcer(gameData, disclosure, index=0, buyer=1, reinforcer=1, numStates = numStates, algo='DDPG',
                     algoArgs = [numStates, gamma, lr_mu, lr_q, tau, batch_size,
                                 buffer_limit, wait_period, grad_steps, reward_norm, std_init, std_decay, std_min]),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0)]
sellers = [ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0)]
log = Log(gameData, disclosure)
rnd = 0

In [3]:
print_interval = 1
for rnd in range(numRounds):
    startRounds(gameData, log, buyers, sellers, rnd)
    #print(log.roundData.buyerValues.item())
    score1, score2 = 0.0, 0.0
    ss1, ss2 = 0.0, 0.0
    for period in range(numPeriods):
        startPeriods(buyers, sellers)
        period_bids = []
        for step in range(numSteps):
            # start
            startSteps(buyers, sellers)

            # set state
            buyers[0].state = generateState(buyers[0], disclosure, depth)

            # trade
            bids, asks = collectOffers(buyers, sellers)
            period_bids.append(np.round(bids[0],1))
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            if currentBidIdx == 0:
                score1 += bprofit
                ss1 += bprofit
            elif currentBidIdx == 1:
                score2 += bprofit
                ss2 += bprofit
                
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose())
            endSteps(buyers, sellers) # updates period counters

            buyers[0].newState  = generateState(buyers[0], disclosure, depth)

            if step == numSteps - 1:
                buyers[0].done = True
            else:
                buyers[0].done = False

            agentsObserve(buyers, sellers)
            agentsTrain(buyers, sellers)

        endPeriods(buyers, sellers)

        if period%100==0:
            print('\t',ss1/100, ss2/100,  period_bids)
            ss1 = 0
            ss2 = 0
        if period%print_interval==0 and period > 2:
            print(step, round(score1 / print_interval, 2), round(score2 / 100, 2), period_bids,
                  np.round(buyers[0].algo.mean.item(),1), np.round(buyers[0].algo.std,1), np.round(buyers[0].algo.action.item(),1))
            score1 = 0.0
            score2 = 0.0
    endRounds(buyers, sellers)

	 0.6630392178113007 0.5630152003965089 [50.5, 35.3, 82.5, 83.8, 63.8, 84.9, nan, nan, nan, nan]
9 229.98 2.4 [64.7, 59.1, 43.6, 65.5, 37.5, 44.5, 62.7, 86.7, 87.9, 38.9] -0.9 0.3 -0.9
9 39.3 0.92 [69.6, 89.5, 76.3, 56.4, 69.8, 40.0, 42.9, 73.8, 89.2, 41.1] -0.8 0.3 -0.8
9 40.18 0.32 [76.0, 35.7, 88.3, 37.5, 60.5, 52.0, 68.7, 89.3, 57.8, 77.3] 0.5 0.3 0.5
9 54.13 0.69 [47.6, 63.6, 76.5, 56.6, 82.4, 62.1, 42.6, 45.6, 46.3, 83.0] 0.7 0.3 0.7
9 63.64 0.67 [55.4, 74.2, 49.8, 67.6, 58.7, 68.3, 44.5, 88.3, 66.4, 41.3] -0.8 0.3 -0.8
9 51.59 0.54 [53.5, 76.8, 41.8, 42.7, 54.7, 76.1, 41.4, 84.2, 66.3, 42.0] -0.7 0.3 -0.7
9 51.8 0.81 [70.3, 57.0, 83.0, 68.4, 81.1, 53.4, 54.1, 58.0, 87.9, 38.7] -0.9 0.3 -0.9
9 55.26 0.43 [61.0, 65.8, 57.5, 72.0, 48.4, 84.3, 84.1, 57.5, 61.8, 49.2] -0.5 0.3 -0.5
9 61.84 0.82 [36.0, 46.4, 44.6, 39.6, 66.2, 67.7, 75.1, 50.0, 45.2, 81.7] 0.7 0.3 0.7
9 68.48 0.62 [73.8, 85.8, 37.0, 38.6, 47.8, 89.9, 77.3, 51.3, nan, nan] 0.2 0.3 0.2
9 60.86 0.63 [47.7, 83.6, 79.9, 49.

KeyboardInterrupt: 

In [ ]:
a = (torch.tensor(buyers[0].state )-buyers[0].algo.state_mean)/(torch.sqrt(buyers[0].algo.state_std))
formatted_elements = [f'{val:.4f}' for val in a.tolist()]
formatted_tensor = "[" + ", ".join(formatted_elements) + "]"
print(formatted_tensor)

In [ ]:
(torch.tensor(buyers[0].state )-buyers[0].algo.state_mean)/(torch.sqrt(buyers[0].algo.state_std))

In [ ]:
log.getPeriod(0,period-100)

In [ ]:
log.graphLearning(rolling_window = 500, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'memory' contains your data
memory = pd.DataFrame(buyers[0].algo.memory, columns=['state', 'action', 'reward', 'newState', 'done'])

# Calculate the rolling mean and standard deviation for actions and rewards
window_size = 100  # You can adjust the window size as needed
rolling_action_mean = memory['action'].rolling(window=window_size).mean()
rolling_reward_mean = memory['reward'].rolling(window=window_size).mean()
rolling_action_std = memory['action'].rolling(window=window_size).std()
rolling_reward_std = memory['reward'].rolling(window=window_size).std()

# Plot the rolling means and standard deviations separately for actions and rewards
plt.figure(figsize=(12, 6))

# Plot Rolling Mean and Std for Actions
plt.subplot(2, 1, 1)
plt.plot(rolling_action_mean, label='Rolling Mean of Actions')
plt.fill_between(
    range(len(memory)),
    rolling_action_mean - rolling_action_std,
    rolling_action_mean + rolling_action_std,
    alpha=0.3,
    label='Rolling Std of Actions'
)
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.title('Rolling Mean and Standard Deviation of Actions')

# Plot Rolling Mean and Std for Rewards
plt.subplot(2, 1, 2)
plt.plot(rolling_reward_mean, label='Rolling Mean of Rewards')
plt.fill_between(
    range(len(memory)),
    rolling_reward_mean - rolling_reward_std,
    rolling_reward_mean + rolling_reward_std,
    alpha=0.3,
    label='Rolling Std of Rewards'
)
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.title('Rolling Mean and Standard Deviation of Rewards')

plt.tight_layout()
plt.show()


In [ ]:
log.graphSales(0, period-8, trackBuyersIdx=[0], trackSellersIdx=[])

In [ ]:
log.graphOffers(0, period-8, trackBuyersIdx=[0], trackSellersIdx=[])

In [ ]:
log.graphTraining(maxNumPeriods=period-1, skip = int((period-1)/20), trackBuyersIdx = [0])